# Training BERT for text classification

In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import json

c:\Users\rohit\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
with open("Dataset\Data.json","r") as f:
    data = json.load(f)

In [4]:
user_inputs = []
trigger_labels = []
for item in data:
    user_inputs.append(item["Question"])
    trigger_labels.append(item["Trigger"])


In [5]:
label_to_index = {label: index for index, label in enumerate(set(trigger_labels))}


In [6]:
label_to_index

{'Engine_On()': 0,
 'Rollup_Windows()': 1,
 'Get_Charge()': 2,
 'Get_System_Status()': 3,
 'Unknown()': 4,
 'Check_Doors()': 5,
 'Get_Range()': 6,
 'Unlock_Doors()': 7,
 'Check_Windows()': 8,
 'Lock_Doors()': 9,
 'Rolldown_Windows()': 10,
 'Check_Engine()': 11,
 'Engine_Off()': 12}

In [7]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
num_labels = len(label_to_index)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model.to(device)
input_ids = []
attention_masks = []

for text in user_inputs:
    encoded_dict = tokenizer.encode_plus(
        text, 
        add_special_tokens = True, 
        max_length=128, 
        padding = "max_length", 
        truncation=True, 
        return_attention_mask = True, 
        return_tensors="pt"
    )
    input_ids.append(encoded_dict["input_ids"])
    attention_masks.append(encoded_dict["attention_mask"])


In [9]:
input_ids = torch.cat(input_ids, dim =0).to(device)
attention_masks = torch.cat(attention_masks, dim=0).to(device)
labels = torch.tensor([label_to_index[label] for label in trigger_labels], device=device)


In [10]:
batch_size = 32
data_set = TensorDataset(input_ids, attention_masks, labels)
data_loader = DataLoader(data_set, batch_size=batch_size, shuffle=True)

In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [12]:
num_epochs = 25

In [13]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in data_loader:
        optimizer.zero_grad()
        batch_input_ids, batch_attention_masks, batch_labels = batch
        output = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
        loss = output.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    avg_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}")

Epoch 1/25 - Loss: 2.5728
Epoch 2/25 - Loss: 2.3205
Epoch 3/25 - Loss: 2.2067
Epoch 4/25 - Loss: 2.0357
Epoch 5/25 - Loss: 1.8914
Epoch 6/25 - Loss: 1.7407
Epoch 7/25 - Loss: 1.5688
Epoch 8/25 - Loss: 1.4052
Epoch 9/25 - Loss: 1.2232
Epoch 10/25 - Loss: 1.0709
Epoch 11/25 - Loss: 0.9667
Epoch 12/25 - Loss: 0.8321
Epoch 13/25 - Loss: 0.7145
Epoch 14/25 - Loss: 0.6028
Epoch 15/25 - Loss: 0.5161
Epoch 16/25 - Loss: 0.4461
Epoch 17/25 - Loss: 0.3963
Epoch 18/25 - Loss: 0.3452
Epoch 19/25 - Loss: 0.2989
Epoch 20/25 - Loss: 0.2746
Epoch 21/25 - Loss: 0.2400
Epoch 22/25 - Loss: 0.2134
Epoch 23/25 - Loss: 0.2023
Epoch 24/25 - Loss: 0.1814
Epoch 25/25 - Loss: 0.1760


In [14]:
output_dir = "./trained_Classification_model/"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./trained_Classification_model/tokenizer_config.json',
 './trained_Classification_model/special_tokens_map.json',
 './trained_Classification_model/vocab.txt',
 './trained_Classification_model/added_tokens.json')

### Testing the model 

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the trained model and tokenizer
output_dir = "./trained_Classification_model/"
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Function to predict the trigger label for user input text
def predict_trigger(text):
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt",
    )

    input_ids = encoded_dict["input_ids"]
    attention_mask = encoded_dict["attention_mask"]

    # Perform inference
    model.eval()
    with torch.no_grad():
        output = model(input_ids=input_ids, attention_mask=attention_mask)

    logits = output.logits
    predicted_label_idx = torch.argmax(logits, dim=1).item()

    # Map the predicted index back to the actual trigger label
    index_to_label = {index: label for label, index in label_to_index.items()}
    predicted_label = index_to_label[predicted_label_idx]

    return predicted_label

# Example usage:
user_input = "Can you lock the car doors please?"
predicted_trigger = predict_trigger(user_input)
print("Predicted Trigger:", predicted_trigger)


In [15]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from functions import functions
def load_model_and_tokenizer(model_dir):
    model = BertForSequenceClassification.from_pretrained(model_dir)
    tokenizer = BertTokenizer.from_pretrained(model_dir)
    return model, tokenizer

def classify_input(user_input, model, tokenizer, label_to_index):
    encoded_dict = tokenizer.encode_plus(
        user_input,
        add_special_tokens=True,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt"
    )
    input_ids = encoded_dict["input_ids"]
    attention_mask = encoded_dict["attention_mask"]
    
    with torch.no_grad():
        output = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = output.logits
    
    predicted_label_index = torch.argmax(logits).item()
    predicted_label = [label for label, index in label_to_index.items() if index == predicted_label_index][0]
    
    return predicted_label

if __name__ == "__main__":
    model_dir = "./trained_Classification_model/"
    label_to_index = {'Engine_On()': 0,
 'Rollup_Windows()': 1,
 'Get_Charge()': 2,
 'Get_System_Status()': 3,
 'Unknown()': 4,
 'Check_Doors()': 5,
 'Get_Range()': 6,
 'Unlock_Doors()': 7,
 'Check_Windows()': 8,
 'Lock_Doors()': 9,
 'Rolldown_Windows()': 10,
 'Check_Engine()': 11,
 'Engine_Off()': 12}
    
    model, tokenizer = load_model_and_tokenizer(model_dir)
    
    while True:
        user_input = input("Enter a sentence: ")
        if user_input.lower() == "exit":
            break
        
        predicted_trigger = classify_input(user_input, model, tokenizer, label_to_index)
        print(f"Predicted Trigger: {predicted_trigger}")


Predicted Trigger: Check_Doors()
Predicted Trigger: Unlock_Doors()
Predicted Trigger: Engine_On()
Predicted Trigger: Unlock_Doors()
Predicted Trigger: Get_Charge()
Predicted Trigger: Check_Doors()
Predicted Trigger: Unlock_Doors()
Predicted Trigger: Check_Doors()


In [ ]:
# this one works better
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from functions import functions
def load_model_and_tokenizer(model_dir):
    model = BertForSequenceClassification.from_pretrained(model_dir)
    tokenizer = BertTokenizer.from_pretrained(model_dir)
    return model, tokenizer

def classify_input(user_input, model, tokenizer, data):
    encoded_dict = tokenizer.encode_plus(
        user_input,
        add_special_tokens=True,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt"
    )

    input_ids = encoded_dict["input_ids"]
    attention_mask = encoded_dict["attention_mask"]
    
    with torch.no_grad():
        output = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = output.logits
    
    predicted_label_index = str(torch.argmax(logits).item())
    #print(type(predicted_label_index), predicted_label_index)
    function = data[predicted_label_index]
    functions.getting_function(value=function)
    #print("*",function, type(function))

if __name__ == "__main__":
    model_dir = "./trained_Classification_model/"    
    model, tokenizer = load_model_and_tokenizer(model_dir)
    with open("Label_to_Function.json", "r") as mappings:
        data = json.load(mappings)
    while True:
        user_input = input("Enter a sentence: ")
        if user_input.lower() == "exit":
            break
        classify_input(user_input, model, tokenizer, data)
